In [99]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame

In [100]:
# Parse html table data, return 3 series company name, number of shares and value
def parse_table(content):
    soup = BeautifulSoup(content, "lxml")    
    tables = soup.find_all('table')
    
    data = []
    rows = tables[3].findAll('tr')

    for tr in rows:
        cols = tr.findAll('td')

        # <td>text</td>
        for td in cols:
            text = td.find(text = True)
            if text is not None:
                data.append(text)
    
    company_names = []
    num_shares = []
    values = []

    for i in range(len(data)):
        if data[i] == 'SOLE':
            company_names.append(data[i - 7])
            num_shares.append(data[i - 3])
            values.append(data[i - 4])

    company_names = Series(company_names)
    company_names = company_names.drop(company_names.index[0])
    num_shares = Series(num_shares)
    num_shares = num_shares.drop(num_shares.index[0])
    values = Series(values)
    values = values.drop(values.index[0])
    
    return company_names, num_shares, values

In [101]:
from __future__ import division
# a:old value, b: new value, return float
def change_percent(a, b):
    if a > b :
        return "{0:.2f}".format(((float(a) - float(b)) / float(a)) * 100)
    
    if a == b:
        return 0.0
    
    if a < b:
        return "{0:.2f}".format(((float(b) - float(a)) / float(a)) * 100)

In [102]:
def parse_int(value):
    value = value.replace(",", "")

    return int(value)

In [103]:
def map_company_name_with_numbers(company_names, num_shares):
    i = 0
#     print names.values[i]
    dicts = {}
    
    while i < len(company_names):
        dicts[company_names.values[i]] = parse_int(num_shares.values[i])
        i += 1
        
    return dicts

In [104]:
class QuarterData(object):
    
    def __init__(self, share, value, share_change_percent, value_change_percent):
        self.share = share
        self.value = value
        self.share_change_percent = share_change_percent  # Change variable name, float
        self.value_change_percent = value_change_percent
        
    def __str__(self):
        # Override to print a readable string presentation of your object
        # below is a dynamic way of doing this without explicity constructing the string manually
        return ', '.join(['{key}={value}'.format(key=key, value=self.__dict__.get(key)) for key in self.__dict__])


In [105]:
# global value
company_map = {}

In [106]:
def get_original_map(company_names, share_map, value_map):
    
    for name in company_names:
        share = share_map.get(name)
        value = value_map.get(name)
    
        single_map = {}
        quarter_data = QuarterData(share = share, value = value, share_change_percent = 0.0, value_change_percent = 0.0)
        single_map[0] = quarter_data
        
        company_map[name] = single_map

In [107]:
def get_company_quarter_data(company_names, num_shares, values, cur_index):

    for name in company_names:
        if name in company_map:
            i = cur_index - 1
            has_pre = False
            
            while i >= 0:
                if i in company_map.get(name):
                    pre_share = company_map.get(name).get(i).share
                    pre_value = company_map.get(name).get(i).value
                    new_share = num_shares.get(name)
                    new_value = values.get(name)
                    
                    share_change = change_percent(pre_share, new_share)
                    value_change = change_percent(pre_value, new_value)

                    single_map = QuarterData(share = new_share, value = new_value, share_change_percent=share_change, value_change_percent=value_change)
                    company_map.get(name)[cur_index] = single_map
                    has_pre = True
                    break
                else:
                    i -= 1
            
            if has_pre == False:
                new_share = num_shares.get(name)
                new_value = values.get(name)
                
                single_quarter_data = QuarterData(share = new_share, value = new_value, share_change_percent = 0.0, value_change_percent = 0.0)
                single_map_company = {}
                single_map_company[cur_index] = single_quarter_data
                company_map[name] = single_map_company
        else:
            # ticker is a new one
            new_share = num_shares.get(name)
            new_value = values.get(name)

            single_quarter_data = QuarterData(share = new_share, value = new_value, share_change_percent = 0.0, value_change_percent = 0.0)
            single_map_company = {}
            single_map_company[cur_index] = single_quarter_data
            company_map[name] = single_map_company

In [108]:
display_all_map = []

In [109]:
def display_company_map():
    
    display_company_names = []
    display_share_quarters = {0:[], 1:[], 2:[], 3:[]}
    display_values_quarters = {0:[], 1:[], 2:[], 3:[]}
    
    for name in sorted(company_map):
        display_company_names.append(name)

        quarter = 0

        while quarter < 4:
            if quarter in company_map.get(name):
                num_share = company_map.get(name).get(quarter).share
                value = company_map.get(name).get(quarter).value

                if quarter == 0:
                    share_change = ""
                    value_change = ""
                else:
                    share_change_percent = company_map.get(name).get(quarter).share_change_percent
                    value_change_percent = company_map.get(name).get(quarter).value_change_percent

                    if share_change_percent >= 0:
                        share_change = "+(" + str(share_change_percent) + "%)"
                    else:
                        share_change = "-(" + str(share_change_percent) + "%)"

                    if value_change_percent >= 0:
                        value_change = "+(" + str(value_change_percent) + "%)"
                    else:
                        value_change = "-(" + str(value_change_percent) + "%)"

                display_single_quarter_share = str(num_share) + share_change
                display_single_quarter_value = str(value) + value_change
                display_share_quarters[quarter].append(display_single_quarter_share)
                display_values_quarters[quarter].append(display_single_quarter_value)

            else:
                display_single_quarter_share = "-"
                display_single_quarter_value = "-"
                display_share_quarters[quarter].append(display_single_quarter_share)
                display_values_quarters[quarter].append(display_single_quarter_value)
                
            quarter += 1

    display_company_names = Series(display_company_names)
    display_share_quarters = Series(display_share_quarters)
    display_values_quarters = Series(display_values_quarters)

#     display_all_map.append(display_company_names)
#     display_all_map.append(display_share_quarters)
#     display_all_map.append(display_values_quarters)
    return display_company_names, display_share_quarters, display_values_quarters

In [110]:
url1 = "https://www.sec.gov/Archives/edgar/data/1510589/000090266417003324/xslForm13F_X01/infotable.xml"
url2 = "https://www.sec.gov/Archives/edgar/data/1510589/000090266417004351/xslForm13F_X01/infotable.xml"
url3 = "https://www.sec.gov/Archives/edgar/data/1510589/000090266418001173/xslForm13F_X01/infotable.xml"
url4 = "https://www.sec.gov/Archives/edgar/data/1510589/000090266418003164/xslForm13F_X01/infotable.xml"

query_urls = [url1, url2, url3, url4]
request_results = []

for query_url in query_urls:
    request_result = requests.get(query_url)
    request_results.append(parse_table(request_result.content))

for i in range(len(request_results)):
    request_result = request_results[i]  # company names, number of shares and values
    
    company = request_result[0]
    num_shares = request_result[1]
    values = request_result[2]

    company_share_map = map_company_name_with_numbers(company, num_shares)
    company_value_map = map_company_name_with_numbers(company, values)

    if i == 0:
        get_original_map(company, company_share_map, company_value_map)
    else:
        get_company_quarter_data(company, company_share_map, company_value_map, i)

In [111]:
names, shares, values = display_company_map()

In [112]:

sec_report = pd.concat([names, Series(shares.get(0)), Series(values.get(0)), Series(shares.get(1)), Series(values.get(1))
                       , Series(shares.get(2)), Series(values.get(2))
                       , Series(shares.get(3)), Series(values.get(3))], axis=1)
sec_report.columns = ['Name', 'Original Share','Original Value', 
                      'After 1 Quarter Share', 'After 1 Quarter Value',
                      'After 2 Quarter Share', 'After 2 Quarter Value',
                      'Current Share', 'Current Value']
sec_report

,Name,Original Share,Original Value,After 1 Quarter Share,After 1 Quarter Value,After 2 Quarter Share,After 2 Quarter Value,Current Share,Current Value
0,58 COM INC,5550451,244830,5550451+(0.0%),350455+(43.14%),5550451+(0.0%),397246+(13.35%),5348673+(3.64%),370877+(6.64%)
1,AC IMMUNE SA,550000,4840,-,-,-,-,-,-
2,ACCELERON PHARMA INC,700000,21273,700000+(0.0%),26124+(22.80%),700000+(0.0%),29708+(13.72%),700000+(0.0%),33964+(14.33%)
3,ADAPTIMMUNE THERAPEUTICS PLC,1190476,5345,-,-,-,-,-,-
4,ALIBABA GROUP HLDG LTD,3570449,503076,3570449+(0.0%),616652+(22.58%),2999020+(16.00%),517121+(16.14%),6508438+(117.02%),1207511+(133.51%)
5,AMAZON COM INC,11562,11192,11562+(0.0%),11115+(0.69%),11562+(0.0%),13521+(21.65%),-,-
6,APELLIS PHARMACEUTICALS INC,-,-,-,-,1428571+(0.0%),31000+(0.0%),4328289+(202.98%),95222+(207.17%)
7,ARCUS BIOSCIENCES INC,-,-,-,-,-,-,767128+(0.0%),9390+(0.0%)
8,BAIDU INC,-,-,-,-,-,-,444831+(0.0%),108094+(0.0%)
9,BEIGENE LTD,3720726,167433,3896782+(4.73%),403161+(140.79%),3896782+(0.0%),380794+(5.55%),5472259+(40.43%),841250+(120.92%)
